In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
import numpy as np
import matplotlib

import warnings; warnings.simplefilter('ignore')


In [2]:
path = 'sneaker_metadata.csv'

In [3]:
def CleanData(path):
    data = pd.read_csv(path)
    subset = data.loc[:, ('brand', 'color', 'model', 'name', 'releaseDate', 'sku')]
    
    subset['date'] = subset['releaseDate'].str.split(" ", n = 1, expand = True)[0]
    new = subset["color"].str.split("/", n = 1, expand = True)
    subset['color1'] = new[0]
    subset['color2'] = new[1].str.replace('-', ' ')
    subset['name'] = subset.name.astype(str)
    subset['brand'] = subset.brand.astype(str)
    subset['name'] = subset.name.replace(subset.brand[subset.brand.notnull()], '', regex = True).str.strip()
    subset['name'] = subset.name.replace(subset.color1[subset.color1.notnull()], '', regex = True).str.strip()
    subset['name'] = subset.name.replace(subset.color2[subset.color2.notnull()], '', regex = True).str.strip()
    subset['name'] = subset.name.replace(subset.model[subset.model.notnull()], '', regex = True).str.strip()
    subset['model'] = subset.model.replace(subset.brand[subset.brand.notnull()], '', regex = True).str.strip()
    subset['name'] = subset.name.str.replace('sacai', 'Sacai').str.replace('&#x27;', '')

    collaborators = ['Sacai', 'Fear Of God', 'Undercover', 'OFF-WHITE', 'yeezy', 'Comme Des Garcons', 'JW Anderson', 'Travis Scott']

    pattern = '|'.join(collaborators)

    subset['collab'] = subset.name.str.contains(pattern).astype(int)
    subset['is_retro'] = subset.model.str.contains('Retro').astype(int)
    
    model_df = subset.loc[:, ('brand', 'color1', 'model', 'collab', 'name', 'is_retro', 'releaseDate','sku')]
    
    return(model_df)


In [4]:
data = CleanData(path)

In [21]:
sneaker_transactions = pd.read_csv('sneaker_transactions.csv')

In [22]:
def MergeDataFrame(left, right):
    full_df = pd.merge(left, right, on = 'sku')
    
    full_df.shoeSize = full_df.shoeSize.astype(str)
    full_df = full_df[~full_df.shoeSize.str.contains("Y")]
    full_df = full_df[~full_df.shoeSize.str.contains('K')]
    full_df.shoeSize = full_df.shoeSize.astype('float')
    full_df = full_df[(full_df['shoeSize'] >= 4.0) & (full_df['shoeSize'] <= 15.0)]
    full_df.shoeSize = full_df.shoeSize.astype('category')
    
    full_df.color1 = full_df.color1.str.lower()
    
    return(full_df)

In [23]:
merged_df = MergeDataFrame(data, sneaker_transactions)

In [25]:
unique_models = merged_df.sku.value_counts().rename_axis('unique_values').reset_index(name='counts')
thousand_transactions = unique_models[unique_models['counts'] >= 1000]

In [26]:
subbed_df = merged_df[merged_df['sku'].isin(thousand_transactions.unique_values)]

In [29]:
sneaker_colors = pd.read_csv('sneaker_colors_gvision.csv')

In [30]:
sneaker_colors.head()

,Unnamed: 0,sku,colors
0,0,c3328941-e1cc-45be-8607-cb680186dc5e,indianred
1,1,e8a9fd74-4cbd-4463-998f-22b865c8794f,darkslategrey
2,2,beec84b5-7cb1-49d7-a942-e08aa4995b56,black
3,3,f949df96-5b8d-4012-93c5-e44af694b06b,silver
4,4,02ff6c84-99bc-46ca-854d-1e5062b8ad1e,brown


In [32]:
df = subbed_df.merge(sneaker_colors, how='left', on='sku')

In [34]:
df.color1[df.color1.str.contains('white')] = 'white'
df.color1[df.color1.str.contains('black')] = 'black'
df.color1[df.color1.str.contains('blue')] = 'blue'
df.color1[df.color1.str.contains('green')] = 'green'
df.color1[df.color1.str.contains('yellow')] = 'yellow'
df.color1[df.color1.str.contains('grey')] = 'grey'
df.color1[df.color1.str.contains('gray')] = 'grey'
df.color1[df.color1.str.contains('silver')] = 'silver'
df.color1[df.color1.str.contains('red')] = 'red'
df.color1[df.color1.str.contains('purple')] = 'purple'
df.color1[df.color1.str.contains('pink')] = 'pink'
df.color1[df.color1.str.contains('navy')] = 'navy'
df.color1[df.color1.str.contains('orange')] = 'orange'
df.color1[df.color1.str.contains('sail')] = 'white'
df.color1[df.color1.str.contains('brown')] = 'brown'

In [35]:
roygbiv = ['white', 'black', 'blue', 'green', 'yellow', 'grey', 'silver' ,'red', 'purple', 'pink', 'navy', 'orange', 'brown'] 

In [36]:
top_colors = df[df['color1'].isin(roygbiv)]

In [37]:
top_colors.head()

,Unnamed: 0_x,sku,localAmount,createdAt,localCurrency,shoeSize,brand,color1,model,collab,name,is_retro,releaseDate,Unnamed: 0_y,colors
6420,0,b209a35a-bee6-4b30-a8c3-931d429ac4db,179.0,2019-06-16T21:30:09+00:00,USD,11.0,Nike,black,Air VaporMax Plus,0,Triple,0,2018-01-25 23-59-59,144,black
6421,1,b209a35a-bee6-4b30-a8c3-931d429ac4db,180.0,2019-06-16T17:04:27+00:00,USD,12.0,Nike,black,Air VaporMax Plus,0,Triple,0,2018-01-25 23-59-59,144,black
6422,2,b209a35a-bee6-4b30-a8c3-931d429ac4db,184.0,2019-06-16T16:05:25+00:00,USD,8.5,Nike,black,Air VaporMax Plus,0,Triple,0,2018-01-25 23-59-59,144,black
6423,3,b209a35a-bee6-4b30-a8c3-931d429ac4db,200.0,2019-06-16T14:52:33+00:00,USD,6.5,Nike,black,Air VaporMax Plus,0,Triple,0,2018-01-25 23-59-59,144,black
6424,4,b209a35a-bee6-4b30-a8c3-931d429ac4db,179.0,2019-06-16T04:32:41+00:00,USD,9.5,Nike,black,Air VaporMax Plus,0,Triple,0,2018-01-25 23-59-59,144,black


In [47]:
master_df = top_colors.loc[:, ('sku', 'localAmount', 'createdAt', 'shoeSize', 'brand', 'model', 'color1', 'collab', 'is_retro', 'releaseDate', 'colors')]

In [48]:
master_df.createdAt = master_df.createdAt.str.replace('T', ' ')
master_df.createdAt = master_df.createdAt.str.split(' ').str[0]
master_df.releaseDate = master_df.releaseDate.str.split(' ').str[0]

In [49]:
master_df.createdAt = pd.to_datetime(master_df.createdAt)

In [50]:
master_df.releaseDate = pd.to_datetime(master_df.releaseDate, errors='coerce')

In [51]:
master_df['releaseYear'] = master_df.releaseDate.dt.year
master_df['releaseYear'] = master_df.releaseYear.astype('category')

In [52]:
master_df['time_to_sale'] = (master_df.createdAt - master_df.releaseDate).dt.days

In [53]:
master_df['day_of_week'] = master_df['createdAt'].dt.day_name()

In [54]:
master_df['shoeSize'] = master_df.shoeSize.astype('category')

In [55]:
master_df.head()

,sku,localAmount,createdAt,shoeSize,brand,model,color1,collab,is_retro,releaseDate,colors,releaseYear,time_to_sale,day_of_week
6420,b209a35a-bee6-4b30-a8c3-931d429ac4db,179.0,2019-06-16,11.0,Nike,Air VaporMax Plus,black,0,0,2018-01-25,black,2018,507,Sunday
6421,b209a35a-bee6-4b30-a8c3-931d429ac4db,180.0,2019-06-16,12.0,Nike,Air VaporMax Plus,black,0,0,2018-01-25,black,2018,507,Sunday
6422,b209a35a-bee6-4b30-a8c3-931d429ac4db,184.0,2019-06-16,8.5,Nike,Air VaporMax Plus,black,0,0,2018-01-25,black,2018,507,Sunday
6423,b209a35a-bee6-4b30-a8c3-931d429ac4db,200.0,2019-06-16,6.5,Nike,Air VaporMax Plus,black,0,0,2018-01-25,black,2018,507,Sunday
6424,b209a35a-bee6-4b30-a8c3-931d429ac4db,179.0,2019-06-16,9.5,Nike,Air VaporMax Plus,black,0,0,2018-01-25,black,2018,507,Sunday


In [56]:
df = master_df.dropna()

In [57]:
df.to_csv('cleaned_data.csv')

In [ ]:
day_0 = df[df['time_to_sale'] == 0]

In [ ]:
day_0['localAmount'] = np.log10(day_0['localAmount'])

In [ ]:
model_df = day_0.loc[:, ('localAmount', 'brand', 'color1', 'collab', 'is_retro', 'day_of_week', 'releaseYear')]

In [ ]:
model_df.head()